# Modelling the Mobility of Dislocations with `matscipy`

The dislocation modelling toolkit provided in `matscipy.dislocation` includes functions to assist in characterising the mobility of dislocations in various systems. The two supported mechanisms are glide and kink, which both allow the dislocation to move in the glide direction.

## Dislocation Glide
Dislocation Glide is where the entire dislocation line migrates across a glide barrier to advance the dislocation. These kinds of structures are easy to generate both in cylindrical and in quadrupolar structures.


### Glide in Dislocation Cylinders
As an example of modelling dislocation glide in cylindrical cells, let's look at the Diamond Screw dislocation in Si, using the [Holland and Marder](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.80.746) potential:

In [ ]:
from matscipy.dislocation import get_elastic_constants, DiamondGlideScrew
# the calculator to provide forces and energies from the potential
from matscipy.calculators.manybody.explicit_forms.stillinger_weber import StillingerWeber,\
                                                                Holland_Marder_PRL_80_746_Si
from matscipy.calculators.manybody import Manybody
calc = Manybody(**StillingerWeber(Holland_Marder_PRL_80_746_Si))

# the function accepts any ASE type of calculator
alat, C11, C12, C44 = get_elastic_constants(calculator=calc, symbol="Si", verbose=False)

Si_screw = DiamondGlideScrew(alat, C11, C12, C44, symbol="Si")

Si_screw_bulk, Si_screw_dislo = Si_screw.build_cylinder(radius=20)
Si_screw.view_cyl(Si_screw_dislo)


To generate the iniial and final structures for a glide event, we can use the `build_glide_configurations` function. We can also then use the `ase.neb` tools to smoothly interpolate an approximate glide path:

In [ ]:
bulk, glide_ini, glide_fin = Si_screw.build_glide_configurations(radius=20)

from ase.neb import NEB

nims = 5

glide_neb = NEB([glide_ini.copy() for i in range(nims-1)] + [glide_fin.copy()])

glide_neb.interpolate(method="idpp", apply_constraint=True)

To visualise the glide structures, we will combine ase's `plot_atoms` to convert a structure to a matplotlib plot, and then use FuncAnimation to animate the glide structures: 

In [ ]:
def animate_glide(images, diamond=True, radii=None):
    from ase.visualize.plot import plot_atoms
    import matplotlib.pyplot as plt
    from matplotlib.animation import FuncAnimation
    from matscipy.utils import get_structure_types
    from visualisation import show_HTML

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.axis("off")

    # Add extra reps of start and end points for clarity
    anim_images = [images[0]] * 3 + images + [images[-1]] * 3

    def plot_frame(framedata):
        ax.clear()
        # Plot an individual frame of the animation 
        framenum, atoms = framedata

        # get CNA colours to enhance plot
        atom_labels, struct_names, colors = get_structure_types(atoms, 
                                                                diamond_structure=diamond)
        atom_colors = [colors[atom_label] for atom_label in atom_labels]

        plot_atoms(atoms, ax=ax, colors=atom_colors, radii=radii)


    animation = FuncAnimation(fig, plot_frame, frames=enumerate(anim_images),
                                save_count=len(anim_images),
                                init_func=lambda: None,
                                interval=200)
    
    # Need to convert animation to HTML in order for it to be visible on the docs
    return show_HTML(animation)

animate_glide(glide_neb.images, radii=0.3)

This is also possible in the dissociated case:

In [ ]:
bulk, glide_ini, glide_fin = Si_screw.build_glide_configurations(radius=20, partial_distance=5)

nims = 5

glide_neb = NEB([glide_ini.copy() for i in range(nims-1)] + [glide_fin.copy()])

glide_neb.interpolate(method="idpp", apply_constraint=True)

animate_glide(glide_neb.images, radii=0.3)

### Glide in Dislocation Quadrupoles
As quadrupoles are extremely useful for modelling dislocations using plane-wave DFT, it can be convenient to be able to set up initial guesses for complex processes such as dislocation glide. In this scenario, we assume that the full infinite dislocation line glides in unison, ignoring the true "kink" process.

We can use the function `build_glide_quadrupoles` to construct a set of images for this system, which can optionally model the glide of either the "left" ($+\mathbf{b}$) or "right" ($-\mathbf{b}$) dislocation cores, or both at once.`

In [ ]:
from matscipy.dislocation import Quadrupole

Si_quad = Quadrupole(DiamondGlideScrew, alat, C11, C12, C44, symbol="Si")

num_images = 5

glide_quads = Si_quad.build_glide_quadrupoles(nims=num_images, 
                                            glide_left=True, # Allow left dislocation glide
                                            glide_right=True, # Allow right dislocation glide
                                            glide_separation=6,
                                            verbose=False)

animate_glide(glide_quads)

In [ ]:
num_images = 5

glide_quads = Si_quad.build_glide_quadrupoles(nims=num_images, 
                                            glide_left=False, # Prevent left dislocation glide
                                            glide_right=True, # Allow right dislocation glide
                                            partial_distance=2, # Dissociated Glide
                                            glide_separation=6,
                                            verbose=False)

animate_glide(glide_quads)

## Dislocation Kink
Dislocation kink is often the preferred mechanism for migration of the dislocation line. It involves a short segment of the dislocation line hopping by one glide vector, which then provides a low barrier for the rest of the dislocation line to migrate.

Here the space of structures to explore is greater, due to the possibility of entire networks of kinks forming, and also due to the possibility for the dislocation to kink out by more than one glide.

### Kink maps
Dislocation kink in `matscipy` is controlled by a kink map, which controls the position of the dislocation line as a function of the vertical coordinate. The kink map is a list of integers, where the values give a line position in units of glide distances, and each element corresponds to an extra cell in z.

A kink map of `[0, 0, 1, 1]` means that the dislocation line initially starts at position zero for two repetitions, and then kinks out by one glide distance for two repetitions.

Both dislocation cylinders and quadrupoles have support for this kind of kink map, but the boundaries are treated differently. In dislocation cylinders (using `build_kink_cyl`), the dislocation line is wrapped back around, and so there is an extra kink back to position zero in the generated structures. In quadrupoles (using `build_kink_quadrupole_network`), the cell is modified such that the dislocation position at the top of the cell becomes the new dislocation line position at the bottom of the cell. Therefore, in dislocation qudrupoles there would be no boundary.

Since the kink map is just a list of integers, it can be more convenient to exploit the concatenation overloading of list addition, and specify kink maps in a form similar to `[0] * 5 + [1] * 5`.

### Kinks in cylinders

In [ ]:
Si_kink = Si_screw.build_kink_cyl(
                    radius=20,
                    kink_map= [0] * 5 + [1] * 5
                )

Si_screw.view_cyl(Si_kink)

### Kink in quadrupoles

In [ ]:
Si_quad_kink = Si_quad.build_kink_quadrupole_network(
    glide_separation=8,
    kink_map=[0]*5 + [1]*5
)

Si_quad.view_quad(Si_quad_kink)